In [ ]:
import init_db

neo4jdb = init_db.Neo4jDB("bolt:0.0.0.0:7687", "neo4j", "1qaz@WSX")
neo4jdb.connect()

In [40]:
shortestPath = neo4jdb.run_query("""MATCH (s1:sample {PATIENT_ID: 'MB-0422'}),
      (s2:sample {PATIENT_ID: 'MB-0420'}),
      p = shortestPath((s1)-[*]-(s2))
WHERE length(p) > 1
RETURN p """)

nodes = None
for record in shortestPath:
    nodes = record["p"].relationships

print(nodes)


(<Relationship id=67 nodes=(<Node id=151 labels=frozenset({'sample'}) properties={'HER2_SNP6': 'NEUT', 'HER2_STATUS': '-', 'AGE_AT_DIAGNOSIS': '66.72', 'NPI': '5.032', 'CELLULARITY': 'high', 'ONCOTREE_CODE': 'IDC', 'HORMONE_THERAPY': 'YES', 'LATERALITY': 'r', 'SAMPLE_TYPE': 'Primary', 'TUMOR_SIZE': '16', 'ER_IHC': 'pos', 'OS_MONTHS': '33.8', 'PR_STATUS': '+', 'CLAUDIN_SUBTYPE': 'LumA', 'OS_STATUS': 'DECEASED', 'BREAST_SURGERY': 'BREAST CONSERVING', 'HISTOLOGICAL_SUBTYPE': 'IDC', 'VITAL_STATUS': 'Died of Other Causes', 'THREEGENE': 'ER+/HER2- Low Prolif', 'ER_STATUS': '+', 'RADIO_THERAPY': 'YES', 'CHEMOTHERAPY': 'NO', 'INFERRED_MENOPAUSAL_STATE': 'post', 'INTCLUST': '8', 'name': 'MB-0422', 'GRADE': '2', 'COHORT': '1', 'PATIENT_ID': 'MB-0422', 'TUMOR_STAGE': '2'}>, <Node id=15941 labels=frozenset({'hugosymbol'}) properties={'name': 'PTEN', 'pagerank': '0.15000000000000002'}>) type='mutation' properties={'RefSeq': 'NM_000314.4', 'Match_Norm_Seq_Allele1': '', 'dbSNP_RS': '', 'Validation_Me

In [41]:
import requests, sys
 
server = "https://rest.ensembl.org"

transcriptIDs = set()

for node in nodes:
    if node['Transcript_ID']:
        transcriptIDs.add(node['Transcript_ID'])

print(transcriptIDs)

{'ENST00000371953', 'ENST00000421745'}


In [42]:
transcript_seq = []
for transcriptID in transcriptIDs:
    uri = server+"/sequence/id/" + transcriptID + "?type=cdna"
    r = requests.get(uri, headers={ "Content-Type" : "text/x-fasta"})
    if not r.ok:
        r.raise_for_status()
        exit()
    transcript_seq.append(r.text)

In [43]:
output_file = open('blast_data/queries/transcript.fsa', 'w')

for seq in transcript_seq:
    output_file.write(seq)

output_file.close()

In [44]:
import os
print(os.system('docker run --rm \
    -v $HOME/Projects/lectures/github_repositories/GitHub/bmi-fmi-2021/SEMESTER_2/Lectures/lecture_2/data_integration/blast_data/blastdb:/blast/blastdb:ro \
    -v $HOME/Projects/lectures/github_repositories/GitHub/bmi-fmi-2021/SEMESTER_2/Lectures/lecture_2/data_integration/blast_data/queries:/blast/queries:ro \
    -v $HOME/Projects/lectures/github_repositories/GitHub/bmi-fmi-2021/SEMESTER_2/Lectures/lecture_2/data_integration/blast_data/results:/blast/results:rw \
    ncbi/blast \
    blastp -query /blast/queries/transcript.fsa -db pdbaa \
    -out /blast/results/blastp.out'))

0
